In [30]:
# Libraries
import os
import re
import json
import time
import random
import pandas as pd

from bs4 import BeautifulSoup

from prettytable import from_csv

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(f'Current working directory: {os.getcwd()}')

Current working directory: /Users/tobiaskoller/Library/CloudStorage/OneDrive-ZHAW/2. Semester/Applied Data Science/DataScience Project/ADS_Project_group3


In [31]:
# Set options for Chrome-Driver (headless mode)
opts = Options()
opts.add_argument("--headless")
#opts.add_argument("user-agent=" + random.choice(ua_list))
#service = Service(executable_path=r'C:\Tools\chromedriver\chromedriver.exe')
service = Service(executable_path=r'/Users/tobiaskoller/Library/CloudStorage/OneDrive-ZHAW/2. Semester/Applied Data Science/tools/chromedriver-mac-x64/chromedriver')

# Chrome-Driver
driver = webdriver.Chrome(service=service, options=opts)

# Base URL of the website to scrape
base_url = 'https://www.immoscout24.ch/de/immobilien/mieten/kanton-zuerich?pn='

# Initialize an empty list to store scraped data
all_roomsLivingSpacePrice = []
all_addresses = []
all_titles = []
all_shortDescriptions = []

# Start with page 1 (or any specific page you want to start with)
current_page = 1

# Specify the maximum number of pages to scrape (you might need to adjust this)
max_pages = 50  # Example: 10 pages

while current_page <= max_pages:
    # Navigate to the next page
    driver.get(base_url + str(current_page))

    # Get the full content of the website
    source = driver.page_source

    # Parse HTML content with BeautifulSoup
    soup = BeautifulSoup(source, 'html.parser')

    # Get number of rooms and square meters and price
    roomsLivingSpacePrice_elements = soup.find_all(class_='HgListingRoomsLivingSpacePrice_roomsLivingSpacePrice_M6Ktp')
    roomsLivingSpacePrice = [element.text for element in roomsLivingSpacePrice_elements]
    all_roomsLivingSpacePrice.extend(roomsLivingSpacePrice)

    # Get square meters
    #squareMeters_elements = soup.find_all(class_='HgListingRoomsLivingSpacePrice_price_u9Vee')
    #squareMeters = [element.text for element in squareMeters_elements]
    #all_squareMeters.extend(squareMeters)

    # Get addresses
    addresses_elements = soup.find_all(class_='HgListingCard_address_JGiFv')
    addresses = [element.text for element in addresses_elements]
    all_addresses.extend(addresses)

    # Get titles
    titles_elements = soup.find_all(class_='HgListingDescription_title_NAAxy')
    titles = [element.text for element in titles_elements]
    all_titles.extend(titles)

    # Get short descriptions
    #short_description_classes = ['HgListingDescription_large_uKs3J', 'HgListingDescription_medium_uKs3J', 'HgListingDescription_medium_NzKMY']
    shortDescriptions_elements = soup.find_all(class_='HgListingDescription_description_r5HCO HgListingDescription_altDescription_Xktod HgListingCard_description_kmLhw')
    shortDescriptions = [element.text for element in shortDescriptions_elements]
    all_shortDescriptions.extend(shortDescriptions)

    # Get prices
    prices_elements = soup.find_all(class_='HgListingRoomsLivingSpacePrice_price_u9Vee')
    prices = [element.text for element in prices_elements]
    all_prices.extend(prices)

    # Increment the page number
    current_page += 1

# Close the driver after scraping is done
driver.close()

# Convert the scraped data to a DataFrame
df = pd.DataFrame({ #'Price': all_prices,
                    'RoomsLivingSpacePrice': all_roomsLivingSpacePrice,
                    'Address': all_addresses,
                    'Title': all_titles,
                    'TitleAndDescription': all_shortDescriptions
                    })

# Split the 'RoomsSquareMetersPrice' column into three separate columns
#df[['Rooms', 'SquareMeters', 'Price']] = df['RoomsSquareMetersPrice'].str.split(' ', 2, expand=True)

csvName = 'immoscout24_zh_20240319.csv'

# Save to file
df.to_csv(csvName, sep=";", index=False)

# Show file
with open(csvName) as table_file:
    tab = from_csv(table_file, delimiter=';')
    
    # Set the alignment for all columns to 'l' (left-align)
    for field_name in tab.field_names:
        tab.align[field_name] = 'l'
    
    print(tab)

+--------------------------------------+-----------------------------------------------------+------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------